In [1]:
###../data/Suicide_Detection.csv
###https://www.opendatabay.com/data/healthcare/9182606a-0bc1-4cb5-8700-73bf7f9fd525

###../data/mendeley/
###https://data.mendeley.com/datasets/zz8j36y24f/1

###../data/reddit_depression_dataset.csv
###https://www.kaggle.com/datasets/rishabhkausish/reddit-depression-dataset/data

import pandas as pd
import pickle
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    average_precision_score,
)
# df = pd.read_csv('../data/Suicide_Detection.csv')
data = pd.read_csv('../data/reddit_depression_dataset.csv',low_memory=False)



In [2]:
df = data.dropna()
df = df[df['subreddit'] != 'depression']
non_suicidal = df[df['label'] == 0] 
suicidal = df[df['label'] == 1]

indices = non_suicidal.sample(n=1331527).index
non_suicidal = non_suicidal.drop(index=indices)

df = pd.concat([non_suicidal,suicidal])

In [ ]:
# harmless_filenames = ['askreddit.csv','books.csv','gaming.csv','jokes.csv','movies.csv']
# harmless = [pd.read_csv(os.path.join('..', 'data', 'mendeley', filename)) for filename in harmless_filenames]
# for harm in harmless:
#     harm['verdict'] = False
# harmless = pd.concat(harmless)
# harmless = harmless.drop('title', axis=1)

# indices = harmless.sample(n=5000).index
# harmless = harmless.drop(index=indices)

# harmful = pd.read_csv('../data/mendeley/suicide_watch.csv')
# harmful['verdict'] = True
# harmful = harmful.drop('title',axis=1)
# df = pd.concat([harmless,harmful]).dropna()

In [6]:
posts = df['body']
# labels = df['class'].apply(lambda x: 1 if x == 'suicide' else 0)
labels = df['label']

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(posts)
pickle.dump(vectorizer, open('vectorizer.p', 'wb'))
y = labels

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=32)
model = LogisticRegression().fit(X_train, y_train)
# model = SVC(C=0.1,gamma=1).fit(X_train, y_train)
# model=SVC()

# params = {'C': [0.1, 1, 10, 100, 1000], 
# 			'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
# 			'kernel': ['linear']} 
# grid = GridSearchCV(estimator=model,param_grid=params)
# grid = grid.fit(X_train,y_train)
# print(grid.best_params_)



y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]
pickle.dump(model, open('model.p', 'wb'))



In [11]:
threshold = 0.7
y_pred = (y_pred_proba >= threshold)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)
avg_precision = average_precision_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall (Sensitivity/True Positive Rate): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")
print(f"Average Precision Score (PR AUC): {avg_precision:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)

Accuracy: 0.9115
Precision: 0.9630
Recall (Sensitivity/True Positive Rate): 0.8542
F1-Score: 0.9053
ROC AUC Score: 0.9110
Average Precision Score (PR AUC): 0.8948

Confusion Matrix:
[[41097  1368]
 [ 6079 35614]]


In [115]:

vectorized = vectorizer.transform(["""I want to die"""])
predict_probas = model.predict_proba(vectorized)
bools = (predict_probas >= threshold)
print('with threshold: ',bools)
print('with threshold: ',predict_probas)
if all(bools[0]):
    if predict_probas[0][0] > predict_probas[0][1]:
        print('with threshold: ', [0])
    else: print('with threshold: ',[1])
elif all(bool == False for bool in bools[0]):
    print('Both False')
elif any(bools[0]):
    if predict_probas[0][0] > predict_probas[0][1]:
        print('with threshold: ',[0])
    else: print('with threshold: ',[1])
print('without threshold: ',model.predict(vectorized))


with threshold:  [[False  True]]
with threshold:  [[0.48824516 0.51175484]]
with threshold:  [1]
without threshold:  [1]
